In [1]:
import os
import csv
import random
import rdflib
from SPARQLWrapper import SPARQLWrapper, JSON, BASIC

In [2]:
def loadAuthFile():
    with open('auth_markov_chain.txt') as f:
        auth =  f.readlines()
        auth = [x.replace('\n','') for x in auth]
    return auth
auth = loadAuthFile()

In [3]:
scene = "scene1"
cs_results = []
with open("../dataset/lancers_task.csv", encoding="utf-8", newline="") as f:
    for cols in csv.reader(f, delimiter=","):
        cs_results.append(cols)
#delete header
cs_results.pop(0)

['taskFormSelectBox2',
 'taskFormSelectBox3',
 'taskFormSelectBox4',
 'taskFormSelectBox5',
 'taskFormSelectBox6',
 'taskFormSelectBox7',
 'taskFormSelectBox9',
 'taskFormSelectBox10',
 'taskFormSelectBox11',
 'taskFormSelectBox12',
 'taskFormSelectBox13',
 'taskFormSelectBox14',
 'taskFormSelectBox16',
 'taskFormSelectBox17',
 'taskFormSelectBox18',
 'taskFormSelectBox19',
 'taskFormSelectBox20',
 'taskFormSelectBox21',
 'taskFormSelectBox23',
 'taskFormSelectBox24',
 'taskFormSelectBox25',
 'taskFormSelectBox26',
 'taskFormSelectBox27',
 'taskFormSelectBox28',
 'taskFormSelectBox30',
 'taskFormSelectBox31',
 'taskFormSelectBox32',
 'taskFormSelectBox33',
 'taskFormSelectBox34',
 'taskFormSelectBox35',
 'taskFormSelectBox37',
 'taskFormSelectBox38',
 'taskFormSelectBox39',
 'taskFormSelectBox40',
 'taskFormSelectBox41',
 'taskFormSelectBox42',
 'taskFormTextArea43']

In [4]:
def convert_name(table):
    result = []
    activity_class_map = {
        "就寝・起床": "BedTimeSleep",
        "飲食": "EatingDrinking",
        "飲食準備": "FoodPreparation",
        "整理・整頓": "HouseArrangement",
        "清掃・洗浄": "HouseCleaning",
        "衛生": "HygieneStyling",
        "娯楽": "Leisure",
        "仕事・学業": "Work",
        "身体活動": "PhysicalActivity",
        "その他": "Other"
    }
    for line in table:
        new_line = []
        for x in line:
            new_line.append(activity_class_map[x])
        result.append(new_line)
    return result

In [5]:
def add_start_end(sequence):
    tmp = ['start']
    tmp.extend(sequence)
    tmp.extend(['end'])
    return tmp

In [6]:
weekdays = []
holidays = []
for x in cs_results:
    weekdays.append(x[0:18])
    holidays.append(x[18:36])
weekdays =  convert_name(weekdays)
holidays =  convert_name(holidays)

In [7]:
weekdays_morning = []
weekdays_afternoon = []
weekdays_evening = []
holidays_morning = []
holidays_afternoon = []
holidays_evening = []
for x in weekdays:
    weekdays_morning.append(add_start_end(x[0:6]))
    weekdays_afternoon.append(add_start_end(x[6:12]))
    weekdays_evening.append(add_start_end(x[12:18]))

for x in holidays:
    holidays_morning.append(add_start_end(x[0:6]))
    holidays_afternoon.append(add_start_end(x[6:12]))
    holidays_evening.append(add_start_end(x[12:18]))

In [8]:
sequence_list = weekdays_morning
sequence_list.extend(weekdays_afternoon)
sequence_list.extend(weekdays_evening)
sequence_list.extend(holidays_morning)
sequence_list.extend(holidays_afternoon)
sequence_list.extend(holidays_evening)

In [9]:
# sequence_list = []
# for x in weekdays:
#     sequence_list.append(add_start_end(x))
# for x in holidays:
#     sequence_list.append(add_start_end(x))

In [10]:
sequence_list

[['start',
  'BedTimeSleep',
  'HygieneStyling',
  'Leisure',
  'FoodPreparation',
  'EatingDrinking',
  'Work',
  'end'],
 ['start',
  'BedTimeSleep',
  'FoodPreparation',
  'EatingDrinking',
  'HygieneStyling',
  'HouseArrangement',
  'Leisure',
  'end'],
 ['start',
  'BedTimeSleep',
  'HouseArrangement',
  'EatingDrinking',
  'HygieneStyling',
  'BedTimeSleep',
  'Other',
  'end'],
 ['start',
  'BedTimeSleep',
  'HygieneStyling',
  'FoodPreparation',
  'EatingDrinking',
  'BedTimeSleep',
  'HouseArrangement',
  'end'],
 ['start',
  'BedTimeSleep',
  'HygieneStyling',
  'FoodPreparation',
  'EatingDrinking',
  'HouseCleaning',
  'Work',
  'end'],
 ['start',
  'BedTimeSleep',
  'HygieneStyling',
  'FoodPreparation',
  'EatingDrinking',
  'HouseArrangement',
  'Work',
  'end'],
 ['start',
  'BedTimeSleep',
  'HygieneStyling',
  'FoodPreparation',
  'EatingDrinking',
  'Work',
  'Work',
  'end'],
 ['start',
  'BedTimeSleep',
  'HygieneStyling',
  'FoodPreparation',
  'EatingDrinking',
 

In [11]:
def create_ngram(sequence_list, n):    
    ngram = {}
    for seq in sequence_list:
        for i in range(n-1):
            if seq[i] in ngram:
                values = ngram[seq[i]]
                if seq[i+1] in values.keys():
                    values[seq[i+1]] += 1
                else:
                    values[seq[i+1]] = 1
                ngram[seq[i]] = values
            else:
                tmp = {}
                tmp[seq[i+1]] = 1
                ngram[seq[i]] = tmp
    return ngram

In [12]:
def create_transition_probability(ngram):    
    transition_probability = {}
    for current_activity in ngram:
        next_activities = ngram[current_activity]
        num = 0
        for na_key in next_activities:
            num += next_activities[na_key]
        probability = {}
        for na_key in next_activities:
            probability[na_key] = next_activities[na_key] / num
        transition_probability[current_activity] =probability
    return transition_probability

In [13]:
def markov_chain(transition_probability):    
    current_activity = "start"
    activity_list = []
    for i in range(6):
        next_candidates = {}
        next_candidates = transition_probability[current_activity]

        keys = []
        values = []
        for x in next_candidates:
            keys.append(x)
            values.append(next_candidates[x])
        next_activity = random.choices(
            population=keys,
            weights=values
        )
        current_activity = next_activity[0]
        if current_activity == "end":
            break
        activity_list.append(current_activity)
    return activity_list

In [14]:
ngram = create_ngram(sequence_list, 6)
transition_probability = create_transition_probability(ngram)
routine_list = []
i=0
while i < 20:
    mc = markov_chain(transition_probability)
    if len(mc) == 6:
        routine_list.append(mc)
        i += 1

In [15]:
routine_list

[['FoodPreparation',
  'EatingDrinking',
  'Work',
  'FoodPreparation',
  'EatingDrinking',
  'HygieneStyling'],
 ['BedTimeSleep',
  'HygieneStyling',
  'FoodPreparation',
  'EatingDrinking',
  'HouseCleaning',
  'BedTimeSleep'],
 ['Leisure',
  'HygieneStyling',
  'Leisure',
  'FoodPreparation',
  'EatingDrinking',
  'HouseCleaning'],
 ['FoodPreparation',
  'EatingDrinking',
  'Leisure',
  'FoodPreparation',
  'EatingDrinking',
  'HygieneStyling'],
 ['BedTimeSleep',
  'HygieneStyling',
  'Leisure',
  'PhysicalActivity',
  'HouseArrangement',
  'FoodPreparation'],
 ['FoodPreparation',
  'EatingDrinking',
  'HouseCleaning',
  'HygieneStyling',
  'Work',
  'Work'],
 ['BedTimeSleep',
  'HygieneStyling',
  'HouseArrangement',
  'HygieneStyling',
  'FoodPreparation',
  'EatingDrinking'],
 ['FoodPreparation',
  'EatingDrinking',
  'Leisure',
  'PhysicalActivity',
  'HygieneStyling',
  'Leisure'],
 ['Work', 'Work', 'Work', 'Work', 'HouseArrangement', 'Leisure'],
 ['FoodPreparation',
  'EatingD

In [16]:
rdf_g = rdflib.Graph()
rdf_g.parse("../ontology/vh2kg_ontology.ttl", format="ttl")

<Graph identifier=Nd192f8715f7b4c00b736810e3828ad93 (<class 'rdflib.graph.Graph'>)>

In [17]:
def get_activity_from_ontology(activity_type, auth, scene):
    result = None
    queryString = """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX : <http://www.owl-ontologies.com/VirtualHome.owl#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        select distinct ?label where { 
            ?activity a :%s ;
                rdfs:label ?label .
         } ORDER BY RAND() limit 1 
    """
    sparql = SPARQLWrapper(auth[0] + "_event_" + scene)
    sparql.setHTTPAuth(BASIC)
    sparql.setCredentials(auth[1], auth[2])
    sparql.addParameter('infer', 'true')
    

    sparql.setQuery(queryString % activity_type)
    sparql.setReturnFormat(JSON)

    try :
        json = sparql.query().convert()
        bindings = json['results']['bindings']
        if len(bindings) > 0:
            result = bindings[0]
    except  Exception as e:
        print(e.args)
    return result

In [18]:
new_routine_list = []
for routine in routine_list:
    new_routine = []
    for activity in routine:
        binding = get_activity_from_ontology(activity, auth, scene)
        if binding is not None:
            new_routine.append(binding["label"]["value"])
        else:
            #print(activity)
            break
    if len(new_routine) == 6:
        new_routine_list.append(new_routine)
    if len(new_routine_list) == 10:
        break

In [19]:
new_routine_list

[['Make coffee',
  'Entertain',
  'Write an email',
  'Cook some food',
  'Relax on sofa',
  'Wash teeth'],
 ['Relax on sofa',
  'Put on coat',
  'Keep an eye on stove as something is cooking',
  'Breakfast',
  'Wash dirty dishes',
  'Go to sleep'],
 ['Read news',
  'Wash teeth',
  'Watch youtube',
  'Keep an eye on stove as something is cooking',
  'Eat',
  'Wash dirty dishes'],
 ['Make toast',
  'Relax on sofa',
  'Watch youtube',
  'Keep an eye on stove as something is cooking',
  'Entertain',
  'Wash teeth'],
 ['Make coffee',
  'Entertain',
  'Shred receipts',
  'Wash teeth',
  'Write an email',
  'Write an email'],
 ['Go to sleep',
  'Wash face',
  'Wash dishes',
  'Wash teeth',
  'Make toast',
  'Type up document'],
 ['Write an email',
  'Use computer',
  'Write an email',
  'Write an email',
  'Wash dishes',
  'Watch youtube'],
 ['Keep an eye on stove as something is cooking',
  'Relax on sofa',
  'Wash teeth',
  'Keep an eye on stove as something is cooking',
  'Entertain',
  '

In [20]:
def export(routine_list):
     with open("episodes/lancers/episode_list10_scene1_20211126.csv", 'w') as f:
            f.write("text\n")
            for routine in routine_list:
                f.write("%s\n" % ' → '.join(routine))

In [21]:
export(new_routine_list)